In [ ]:
import os, glob, ast, geojson
import scanpy as sc
import seaborn as sns
import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.sparse import csr_matrix
from matplotlib.colors import to_rgb
import anndata as ad
from scipy.sparse import csr_matrix
from pyimzml.ImzMLParser import ImzMLParser

In [ ]:
def cluster_leiden(adata, n_iterations = 2, resolution = 0.5, spot_size = 1, show_plots = True):
    """
    Perform leiden clustering on an AnnData object.
    The argument spot_size describes the resolution in um / how far apart the coordinates are.
    """
    sc.pp.pca(adata, n_comps = 50)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, n_components = 3)
    sc.tl.leiden(adata, key_added = "clusters", flavor = "leidenalg", directed = False, n_iterations = n_iterations, resolution = resolution)
    
    if show_plots:
        #sc.pl.spatial(adata, color = "clusters", img_key = None, spot_size = spot_size)
        sc.pl.umap(adata, color = ["clusters"])

        #Get spatial coordinates
        x = adata.obsm['spatial'][:, 0]
        y = adata.obsm['spatial'][:, 1]

        # Get cluster labels and convert them to numbers
        clusters = adata.obs["clusters"].astype(str).astype("category")
        cluster_labels = clusters.cat.codes  # Convert categories to numerical codes

        # Generate a colormap using Seaborn or Matplotlib
        num_clusters = len(clusters.cat.categories)
        palette = sns.color_palette("tab10", num_clusters)  
        colors = np.array([palette[i] for i in cluster_labels])  
    
        # Scatter plot with square markers
        plt.figure(figsize=(6, 6))
        plt.title("Clusters 030622_B71407")
        plt.scatter(x, y, c=colors, marker="s", s=100)  
        plt.axis("equal") 
        plt.gca().invert_yaxis()
        plt.show()
        
def create_UMAP_in_RGB(adata, show_plots = True):
    """
    Rescale the values of the 3 axes of the UMAP to be shown as RGB colours.
    """

    # rescale using the individual max and min for each colour axis (global max and min gives less varied colours)
    old_min = np.min(adata.obsm['X_umap'], axis = 0)
    old_max = np.max(adata.obsm['X_umap'], axis = 0)
    new_min = 0
    new_max = 1
    umap_RGB = (adata.obsm['X_umap'] - old_min) / (old_max - old_min) * (new_max - new_min)

    # convert the rows to tuples defining a RGB colour
    umap_RGB = [tuple(row) for row in umap_RGB]

    # add the UMAP in RGB scale as observation to adata
    adata.obs['umap_RGB'] = umap_RGB

    if show_plots:
        # plot (with directly the umap_RGB array as colourmap)
        plt.scatter(x = adata.obsm['spatial'][:,0], y = adata.obsm['spatial'][:,1], c = umap_RGB, marker="s", s=50)
        plt.gca().invert_yaxis()
        plt.show()

def filter_m_over_z(adata, min_m_z = 0, max_m_z = 10000):
    """
    Filter the peaks for a certain m/z cutoff. The defaults are chosen so they are outside of the range of m/z in this data.
    """

    adata_m_z = adata[:, (adata.var['m_over_z'] > min_m_z) & (adata.var['m_over_z'] < max_m_z)].copy()

def subcluster_islets(adata, islets_cluster_id):
    """
    Subcluster the pancreatic islet cluster to find morphology
    """
    islets_cluster_id = str(islets_cluster_id)
    sc.tl.leiden(adata, resolution = 0.4, restrict_to = ('clusters',[islets_cluster_id]), key_added ='islets_subcluster')
    
    #sc.pl.spatial(adata, color = "islets_subcluster", img_key = None, spot_size = 1)
    #sc.pl.umap(adata, color = ["islets_subcluster"])

    x = adata.obsm['spatial'][:, 0]
    y = adata.obsm['spatial'][:, 1]
    # Get cluster labels and convert them to numbers
    clusters = adata.obs["islets_subcluster"].astype(str).astype("category")
    cluster_labels = clusters.cat.codes  # Convert categories to numerical codes
    num_clusters = len(clusters.cat.categories)
    palette = sns.color_palette("tab10", num_clusters)  
    colors = np.array([palette[i] for i in cluster_labels])  
    # Scatter plot with square markers
    plt.figure(figsize=(6, 6))
    scatter = plt.scatter(x, y, c=colors, marker="s", s=50) 
    
    # Add a legend
    plt.gca().invert_yaxis()
    plt.axis("equal")  # Ensures proper aspect ratio
    plt.title(f"Subclustering of Islet Cluster {islets_cluster_id}, 030622_B71407")
    plt.show()

    # UMAP Plot
    sc.pl.umap(adata, color=["islets_subcluster"])

In [ ]:
def load_cut_imzml_data(imzml_path: str):
    """
    Create an anndata object from an imzml file
    Remove the outer edges of the image to remove any edge artifacts
    """
    # Parse the imzML file
    parser = ImzMLParser(imzml_path)

    # Create a list to store the spectra and coordinates
    my_spectra = []

    for idx, (x, y, z) in enumerate(parser.coordinates):
        mzs, intensities = parser.getspectrum(idx)
        my_spectra.append([mzs, intensities, (x, y, z)])

    my_spectra = np.array(my_spectra, dtype="object")

    # Extract all coordinates
    coordinates = np.array([spectrum[2] for spectrum in my_spectra])

    # Determine bounds
    x_vals = coordinates[:, 0]
    y_vals = coordinates[:, 1]

    x_min, x_max = x_vals.min(), x_vals.max()
    y_min, y_max = y_vals.min(), y_vals.max()

    # Filter out spectra on the outer edges
    valid_indices = [
        idx for idx, (x, y, z) in enumerate(coordinates)
        if x_min < x < x_max and y_min < y < y_max
    ]

    my_spectra = my_spectra[valid_indices]

    # Extract intensity values and pad
    intensities = [spectrum[1] for spectrum in my_spectra]
    max_len = max(len(spec) for spec in intensities)
    padded_intensities = [np.pad(spec, (0, max_len - len(spec))) for spec in intensities]

    intensity_matrix = csr_matrix(padded_intensities)

    # Create AnnData object
    adata = sc.AnnData(X=intensity_matrix)

    # Add spatial coordinates
    adata.obs['x'] = [coord[0] for _, _, coord in my_spectra]
    adata.obs['y'] = [coord[1] for _, _, coord in my_spectra]
    adata.obs['z'] = [coord[2] for _, _, coord in my_spectra]
    adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()

    # Add m/z values as variables
    mz_values = [spectrum[0] for spectrum in my_spectra]
    adata.var['mz'] = np.array(mz_values[0])  # assuming consistent m/z axis

    return adata


In [ ]:
# lOAD EXAMPLE
path = "/storage/homefs/ha25g949/pannet_metabolism/parallel/rms/total/negative/frequency_0.1/peak_pick_ref/"
file_name = "010622_B11965_TN_Analyte_1_CLMC_1.imzML"
file_path = path + file_name
adata = load_cut_imzml_data(file_path)

In [ ]:
# PLOT EXAMPLE
# normalise and log transfrom
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, inplace = True)
sc.pp.log1p(adata)

# cluster
cluster_leiden(adata, resolution = 0.3)

# make a UMAP in RGB
create_UMAP_in_RGB(adata)